# Question: Write a python script that uses pandas to find the 3 lowest-paid employees who have done at least 10 projects.

## 1.IMPORTING

#### #1 First i imported the data genrator code:

In [168]:
#@title Run this cell to generate the data
! pip install faker
import numpy as np
import pandas as pd
import uuid
import datetime

from faker.providers.person.en import Provider
from faker import Faker

from faker import Faker
fake = Faker()
Faker.seed(0)

n_employees = 50
first_names = pd.Series(list(set(Provider.first_names))).sample(n_employees)
df_employees = pd.DataFrame(columns=["id", "name", "salary"])
df_employees["name"] = (first_names).sample(frac=1).values
mu = 28_000
minimum = 16_000
df_employees["salary"] = (np.random.pareto(a=6, size=(50,1)) * mu) + minimum
df_employees["salary"] = (df_employees["salary"]/100).astype(int) * 100
df_employees["id"] = pd.Series([str(uuid.uuid4()).split('-')[1] for x in range(n_employees)])

def create_random_project(number_of_employees, duration, year, is_ongoing=False):
    df_projects = pd.DataFrame(index=range(number_of_employees), columns=["employee_id", "project_id", "start_date", "end_date"])
    
    employees = df_employees.id.sample(number_of_employees)
    
    start_date = datetime.date(year=year, month=1, day=1)
    end_date = datetime.date(year=year+1, month=1, day=1)
    project_start_date = fake.date_between(start_date=start_date, end_date=end_date)
    project_end_date = project_start_date + datetime.timedelta(days=duration)
    
    if is_ongoing:
        project_end_date = None
        
    project_id = str(uuid.uuid4()).split('-')[4]
    
    df_projects["employee_id"] = employees.values
    df_projects["project_id"] = project_id
    df_projects["start_date"] = project_start_date
    df_projects["end_date"] = project_end_date
    
    
    return df_projects

n_projects = 150
projects = []
for proj in range(n_projects):
    number_of_employees = np.random.choice([1,2,3,4], replace=True, p=[0.25, 0.5, 0.15, 0.1], size=1)
    duration = (np.random.pareto(a=5, size=1) * 30*4) + 14
    year = np.random.choice([2017, 2018, 2019, 2020])
    is_ongoing = bool(np.random.binomial(n=1, p=0.4)) if year >= 2019 else False
    new_project = (create_random_project(number_of_employees=number_of_employees[0], duration=duration[0], year=year, is_ongoing=is_ongoing).values)
    
    projects.append(new_project)
    
projects_data = np.vstack(projects)
df_projects = (
    pd.DataFrame(projects_data, columns=["employee_id", "project_id", "start_date", "end_date"])
    .sort_values(["project_id", "start_date", "end_date"])
    .reset_index(drop=True)
)

#### #2 Then i took a look on the data:

In [169]:
df_employees.head(5)

,id,name,salary
0,87b4,Amy,17700
1,e656,Marilla,19500
2,1331,Achsah,21100
3,2f83,Rosaria,25900
4,0970,Shelley,21000


In [170]:
df_projects.head(5)


,employee_id,project_id,start_date,end_date
0,a804,00a481377f9e,2019-03-20,None
1,b199,00a481377f9e,2019-03-20,None
2,4890,00a481377f9e,2019-03-20,None
3,e975,00a481377f9e,2019-03-20,None
4,b68b,015b88a87534,2019-10-07,2019-12-05


## 2.Cleaning

#### #3 Then i calculated the number of apperance of each Employee ID and added it to the "df_employees" Dataframe:

In [171]:
df_projects['employee_id'].value_counts()

a804    12
9d75    11
a46d    11
85c0    10
0139     9
1fb7     9
5171     9
23e0     9
5c45     9
e50b     9
948c     8
af50     8
050a     8
2301     8
fce1     8
9371     8
4890     8
b199     8
ac8e     7
f937     7
eb7e     7
f4a9     7
90c4     7
0fa2     7
e975     7
b68b     7
4747     7
f41d     7
3128     7
2517     6
aacb     6
1331     5
c781     5
47f0     5
e656     5
6ecb     5
086a     5
197c     5
53ca     5
21c5     4
c2bd     4
87b4     4
f4ed     4
2f83     3
b448     3
0bb5     3
0970     3
aadd     2
5208     2
4c9d     2
Name: employee_id, dtype: int64

In [172]:
#assigned new varibale "amount_projects" to store the counts
amount_projects=df_projects["employee_id"].value_counts()

In [173]:
#i set a name for the varibal to appere in the dataframe as a column
amount_projects.name="amount_projects"

In [174]:
# i set the index as "id"
df_employees.set_index("id", drop=True, inplace=True)

In [175]:
#i added the new column to the df_employees dataframe
df_employees=df_employees.join(amount_projects, how="left")

In [176]:
#printed the dataframe to check
df_employees.head()

,name,salary,amount_projects
id,,,
87b4,Amy,17700,4
e656,Marilla,19500,5
1331,Achsah,21100,5
2f83,Rosaria,25900,3
0970,Shelley,21000,3


In [177]:
#i reset the index to prepare the dataframe for merging
df_employees.reset_index

<bound method DataFrame.reset_index of            name  salary  amount_projects
id                                      
87b4        Amy   17700                4
e656    Marilla   19500                5
1331     Achsah   21100                5
2f83    Rosaria   25900                3
0970    Shelley   21000                3
5c45     Benita   18900                9
aadd     Clemie   21100                2
0139      Mills   16800                9
197c       Vere   32400                5
af50      Craig   28600                8
9d75     Scotty   16700               11
21c5    Delfina   26500                4
4c9d     Elisha   17700                2
c2bd      Murry   19700                4
a804      Kyrie   29200               12
0fa2     Chanie   25200                7
4747    Aracely   16600                7
85c0       Erin   18900               10
b199      Devin   17200                8
e975  Christine   18600                7
2301     Huldah   18000                8
47f0     Jazmyn   

#### #4 i merged the two datasets into "df" and prepared it for the next step:

In [178]:
#renamed the employeee_id colmun to id so i can merge by id column
df_projects.rename(columns = {'employee_id':'id'}, inplace = True)

In [179]:
#merged the two dataframes
df = pd.merge(df_employees, df_projects, on="id")

In [180]:
#before dropping
#checking for duplicates to remove the duplicated ids
df.id.duplicated().sum()

275

In [181]:
#dropped the duplicates
df=df.drop_duplicates('id')
df.head()

,id,name,salary,amount_projects,project_id,start_date,end_date
0,87b4,Amy,17700,4,054b523a576e,2017-09-21,2017-10-08
4,e656,Marilla,19500,5,8a32f36026e5,2018-11-18,2018-12-24
9,1331,Achsah,21100,5,3c912604255e,2018-12-13,2019-02-10
14,2f83,Rosaria,25900,3,045f339fb707,2020-04-30,2020-05-16
17,0970,Shelley,21000,3,a6fb65cd632c,2017-09-13,2017-09-27


In [182]:
#aftrer dropping
df.id.duplicated().sum()

0

In [183]:
#dropping the not needed columns
df=df.drop(['project_id', 'start_date','end_date'], axis=1)
df.head(5)

,id,name,salary,amount_projects
0,87b4,Amy,17700,4
4,e656,Marilla,19500,5
9,1331,Achsah,21100,5
14,2f83,Rosaria,25900,3
17,0970,Shelley,21000,3


## 3.Sorting

##### #5 i filtered the dataframe by number of projects to leave only employees with 10 or more projects:

In [184]:
more_than_10=df[df['amount_projects']>=10]
more_than_10

,id,name,salary,amount_projects
53,9d75,Scotty,16700,11
74,a804,Kyrie,29200,12
100,85c0,Erin,18900,10
226,a46d,Holli,16500,11


##### #6 finally i sorted the remining employees by salary to see the lowest paid employees with 10 or more projects:

In [185]:
more_than_10.sort_values("salary")

,id,name,salary,amount_projects
226,a46d,Holli,16500,11
53,9d75,Scotty,16700,11
100,85c0,Erin,18900,10
74,a804,Kyrie,29200,12
